In [24]:
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [25]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from configs import g_key

In [26]:
#Load csv in part 1 to a dataframe

#clean city data filepath
path = "../output_data/clean_city_data.csv"

city_df = pd.read_csv(path)
city_df = city_df.dropna()
city_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,vaini,-21.20,-175.20,71.600,88,20,5.816034,TO,1593094895
1,plettenberg bay,-34.05,23.37,75.416,34,0,6.487115,ZA,1593094895
2,lakhtar,22.85,71.78,94.442,47,100,6.151575,IN,1593094895
3,new norfolk,-42.78,147.06,46.004,79,0,5.010737,AU,1593094895
4,hithadhoo,-0.60,73.08,84.668,72,83,10.312276,MV,1593094652


# Humidity Heatmap

In [27]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [28]:
#Store Lat and Lng into locatins
locations = city_df[["Lat", "Lng"]]

#Store humidity into humidity
humidity = city_df["Humidity"].astype(float)

In [29]:
#Create humidity Heatmap layer
fig = gmaps.figure()

humid_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False,
                                 max_intensity = 100, point_radius = 1)

fig.add_layer(humid_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [36]:
#Narrow down cities to fit weather conditions

#Filter cities with max temps between 60 and 90 degrees
comfortable_df = city_df.loc[(city_df["Max Temp"] > 70) & (city_df["Max Temp"] < 85), :]
#filter by humidity below 80 percent
comfortable_df = comfortable_df.loc[(comfortable_df["Humidity"] < 40), :]
#filter by wind speed below 10 mph
comfortable_df = comfortable_df.loc[(comfortable_df["Wind Speed"] < 5), :]
#filter by cloudiness below 50 %
comfortable_df = comfortable_df.loc[(comfortable_df["Cloudiness"] < 40), :]

#drop rows with na values
comfortable_df = comfortable_df.dropna()
comfortable_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,port alfred,-33.59,26.89,80.006,29,0,1.006621,ZA,1593094902
219,maun,-19.98,23.42,75.200,29,20,2.236936,BW,1593094915
270,saint george,37.10,-113.58,77.000,19,1,3.333035,US,1593094921
281,shache,38.42,77.24,80.024,16,1,1.543486,CN,1593094922
390,mwinilunga,-11.74,24.43,81.986,22,1,0.850036,ZM,1593094933
418,menongue,-14.66,17.69,79.952,16,0,4.227810,AO,1593094937
468,komatipoort,-25.43,31.95,73.400,31,0,3.355404,ZA,1593094943
472,shakawe,-18.37,21.85,77.648,18,0,4.540981,BW,1593094944
484,xinzhou,38.41,112.73,72.176,29,0,2.259306,CN,1593094945
504,verkhoyansk,67.54,133.38,73.724,36,0,1.476378,RU,1593094946


In [37]:
#store into variable 'hotel_df'
hotel_df = comfortable_df
#adds column "Hotel_name"
hotel_df["Hotel Name"] = ""

#finding a hotel within 5000 meters of our most comfortable cities
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    lat = row.Lat
    lng = row.Lng
    params = {
        "location": f'{lat}, {lng}',
        "radius": 5000,
        "key" : g_key,
        "type" : "lodging"
    }
    response = requests.get(base_url, params = params).json()
    #updates df with the hotel name
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
    #displays message about the issue city and adds "NA" to hold the Hotel name's place
    except:
        print(f'Issues with finding a hotel at {row["City"]}')
        hotel_df.loc[index, "Hotel Name"] = "NA"
        
hotel_df

Issues with finding a hotel at verkhoyansk


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
74,port alfred,-33.59,26.89,80.006,29,0,1.006621,ZA,1593094902,The Halyards Hotel
219,maun,-19.98,23.42,75.200,29,20,2.236936,BW,1593094915,Enviro Villa
270,saint george,37.10,-113.58,77.000,19,1,3.333035,US,1593094921,Best Western Plus Abbey Inn
281,shache,38.42,77.24,80.024,16,1,1.543486,CN,1593094922,Delong Hotel
390,mwinilunga,-11.74,24.43,81.986,22,1,0.850036,ZM,1593094933,Kwakuwahi Lodge
418,menongue,-14.66,17.69,79.952,16,0,4.227810,AO,1593094937,Ritz Lauca
468,komatipoort,-25.43,31.95,73.400,31,0,3.355404,ZA,1593094943,Kruger View Backpackers
472,shakawe,-18.37,21.85,77.648,18,0,4.540981,BW,1593094944,Ditlhapi Guest House
484,xinzhou,38.41,112.73,72.176,29,0,2.259306,CN,1593094945,Shell Hotel - Xinzhou
504,verkhoyansk,67.54,133.38,73.724,36,0,1.476378,RU,1593094946,NA


In [38]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
74,port alfred,-33.59,26.89,80.006,29,0,1.006621,ZA,1593094902,The Halyards Hotel
219,maun,-19.98,23.42,75.200,29,20,2.236936,BW,1593094915,Enviro Villa
270,saint george,37.10,-113.58,77.000,19,1,3.333035,US,1593094921,Best Western Plus Abbey Inn
281,shache,38.42,77.24,80.024,16,1,1.543486,CN,1593094922,Delong Hotel
390,mwinilunga,-11.74,24.43,81.986,22,1,0.850036,ZM,1593094933,Kwakuwahi Lodge
418,menongue,-14.66,17.69,79.952,16,0,4.227810,AO,1593094937,Ritz Lauca
468,komatipoort,-25.43,31.95,73.400,31,0,3.355404,ZA,1593094943,Kruger View Backpackers
472,shakawe,-18.37,21.85,77.648,18,0,4.540981,BW,1593094944,Ditlhapi Guest House
484,xinzhou,38.41,112.73,72.176,29,0,2.259306,CN,1593094945,Shell Hotel - Xinzhou


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

#Add markers on top of the heatmap
hotel_layer = gmaps.symbol_layer(
    locations, fill_color = "blue",
    stroke_color = 'black', scale = 2,
    info_box_content = hotel_info)

fig.add_layer(hotel_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))